# Classification using milliGrad
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mdehling/milligrad/blob/main/demo/classification.ipynb)

Make sure you have the `milliGrad` package installed.

* If you are running this notebook on Google Colab, the first cell will take
  care of it for you.
* If you opened this notebook from within GitHub Codespaces, `milliGrad` should
  already be installed.

In [ ]:
try:
    import milligrad
    MISSING_MILLIGRAD = False
except ImportError:
    MISSING_MILLIGRAD = True

try:
    from google import colab
    RUNNING_ON_COLAB = True
except ImportError:
    RUNNING_ON_COLAB = False

if MISSING_MILLIGRAD and RUNNING_ON_COLAB:
    !pip install -q 'git+https://github.com/mdehling/milligrad.git'
elif MISSING_MILLIGRAD and not RUNNING_ON_COLAB:
    raise ModuleNotFoundError("please install 'milligrad' package")

In [ ]:
from milligrad import Tensor, nn
from milligrad.nn import functional as F
from milligrad.datasets import make_moons, make_blob_circle

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('image', cmap='brg')


## Binary Classification
In this section we build a simple neural network for binary classification and
train it on the moons dataset using stochastic gradient descent to minimize
binary crossentropy loss.

In [ ]:
x, y = make_moons(100, noise=0.1)
x, y = Tensor(x, _label='x'), Tensor(y, _label='y')

In [ ]:
model = nn.Sequential([
    nn.Linear(2, 32),
    nn.ReLU(),
    nn.Linear(32, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
])
opt = nn.optim.SGD(model, lr=1e-1)

for i in range(100):
    opt.zero_grad()
    loss = F.binary_cross_entropy(model(x), y)
    loss.backward()
    opt.step()

accuracy = np.mean( (model(x).value >= 0.5) == (y.value >= 0.5) )
print(f"Classification Accuracy: {int(accuracy*100)}%")

In [ ]:
xx, yy = np.meshgrid(np.linspace(-2.0, 2.0, 41), np.linspace(-1.5, 1.5, 31))

X = np.stack([xx.flatten(),yy.flatten()], axis=-1)
Y = model(X).value
Z = Y.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.3, levels=1)
plt.scatter(x.value[:,0], x.value[:,1], c=y.value)
plt.show()

## Multiclass Classification
This is a simple example of a multiclass classification task.  We train a neural
network similar to the one used for binary classification, but with the final
layers replaced to handle multiple classes.

In [ ]:
num_classes = 5
x, y = make_blob_circle(num_classes, n=num_classes*50)
x, y = Tensor(x, _label='x'), Tensor(y, _label='y')

In [ ]:
def cross_entropy(y_pred, y_true):
    return - (y_true * y_pred).mean()

model = nn.Sequential([
    nn.Linear(2, 32),
    nn.ReLU(),
    nn.Linear(32, 32),
    nn.ReLU(),
    nn.Linear(32, num_classes),
    nn.LogSoftmax()
])
opt = nn.optim.SGD(model, lr=1e-1)

for i in range(250):
    opt.zero_grad()
    loss = cross_entropy(model(x), y)
    loss.backward()
    opt.step()

accuracy = np.mean( model(x).value.argmax(axis=-1) == y.value.argmax(axis=-1) )
print(f"Classification Accuracy: {int(accuracy*100)}%")

In [ ]:
xx, yy = np.meshgrid(np.linspace(-2.0, 2.0, 41), np.linspace(-1.5, 1.5, 31))

X = np.stack([xx.flatten(),yy.flatten()], axis=-1)
Y = model(X).value.argmax(axis=-1)
Z = Y.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.3, levels=num_classes-1)
#plt.pcolormesh(xx, yy, Z, alpha=0.3)
plt.scatter(x.value[:,0], x.value[:,1], c=y.value.argmax(axis=-1))
plt.xlim(-2, 2)
plt.ylim(-1.5, 1.5)
plt.show()